In [14]:
import copy
import os
import argparse
import torch
import numpy as np
import random

%load_ext autoreload
%autoreload 2
from erasure.utils.config.local_ctx import Local
from erasure.utils.config.global_ctx import Global 
from erasure.core.factory_base import ConfigurableFactory
from erasure.data.datasets.DatasetManager import DatasetManager

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
config_file = os.path.join("configs", "survey", "spotify_tracks.jsonc")
#config_file = os.path.join("configs", "example.jsonc")


global_ctx = Global(config_file)
global_ctx.factory = ConfigurableFactory(global_ctx)

#Create Dataset
data_manager = global_ctx.factory.get_object( Local( global_ctx.config.data ))
global_ctx.dataset = data_manager

2025-02-01 22:26:10,-1016584700 | INFO | 941989 - Creating Global Context for: configs/survey/spotify_tracks.jsonc
2025-02-01 22:26:10,-1016584648 | INFO | 941989 - Setting seeds to: 0
2025-02-01 22:26:10,-1016584643 | INFO | 941989 - Caching System: True.
2025-02-01 22:26:10,-1016584643 | INFO | 941989 - Created Configurable: erasure.data.preprocessing.reshape_x_z.reshape_x_z
2025-02-01 22:26:10,-1016584642 | INFO | 941989 - Created Configurable: erasure.data.data_sources.HFDataSource.SpotifyHFDataSource
2025-02-01 22:26:10,-1016584641 | INFO | 941989 - {'class': 'erasure.data.data_sources.HFDataSource.SpotifyHFDataSource', 'parameters': {'path': 'maharshipandya/spotify-tracks-dataset', 'data_columns': ['artists', 'popularity', 'duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature'], 'to_encode': ['artists', 'popularity', 'explicit', 'track_genre'], 'configurati

Filtering Data:   0%|          | 0/12 [00:01<?, ?it/s]

2025-02-01 22:26:16,-1016578364 | INFO | 941989 - ['all', 'all_shuffled', '-', 'forget', 'other_ids']
2025-02-01 22:26:16,-1016578363 | INFO | 941989 - Instantiating: erasure.data.datasets.DataSplitter.DataSplitterPercentage
2025-02-01 22:26:16,-1016578361 | INFO | 941989 - ['all', 'all_shuffled', '-', 'forget', 'other_ids', 'retain', 'test']
2025-02-01 22:26:16,-1016578359 | INFO | 941989 - Instantiating: erasure.data.datasets.DataSplitter.DataSplitterConcat
2025-02-01 22:26:16,-1016578352 | INFO | 941989 - ['all', 'all_shuffled', '-', 'forget', 'other_ids', 'retain', 'test', 'train']
2025-02-01 22:26:16,-1016578321 | INFO | 941989 - ['all', 'all_shuffled', '-', 'forget', 'other_ids', 'retain', 'test', 'train']
2025-02-01 22:26:16,-1016578290 | INFO | 941989 - Created Configurable: erasure.data.datasets.DatasetManager.DatasetManager


In [18]:
data_manager.n_classes

114

In [27]:
forget_loader, _ = data_manager.get_loader_for('forget')
len(forget_loader)

0

In [28]:
train_loader, _ = data_manager.get_loader_for('train')

2025-02-01 22:27:12,-1016522561 | INFO | 941989 - TRAINING WITH 91208 samples


In [29]:
forget = data_manager.partitions['forget']
len(forget)

43

In [97]:
from transformers import BertTokenizer

# Load the same tokenizer used for tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Iterate through your DataLoader
for X, y in train_loader:
    print(X)

[('Daddy Yankee', 'Slow Pulp', 'Lalo Ebratt;Sebastian Yatra;Yera;Trapical Minds', 'Captain Jack', 'Giacomo Puccini;Luciano Pavarotti;Mirella Freni;Elizabeth Harwood;Rolando Panerai;Berliner Philharmoniker;Herbert von Karajan', 'Norlie & KKV', 'Carl Smith', 'Zedd;Liam Payne', 'Mosaic MSC', 'Eric Prydz;Empire of the Sun', 'Alka Yagnik;Arko', 'Nat King Cole', 'Fall Out Boy', 'Arckanum', 'Ensiferum', 'maNga;Kenan Doğulu', 'White Noise Sleep Sounds;Sleep Sound Library', 'Jamie Miller;salem ilese', 'It Dies Today', 'Manuel Silva', 'Bailey Jehl', 'Robert Earl Keen', 'David Cross', 'Don Omar;Alexis y Fido', "Mo' Horizons;Fort Knox 5", 'KAROL G;Mariah Angeliq', 'Divino Niño', 'Dub Pistols;Seanie Tee', 'Kijugo', 'Fan Yi Chen', 'Freestylers', 'Bruno & Barretto', 'Pearl Jam', 'Faron Young', 'Roberto Firpo;Ignacio Murillo', 'Foreign Fields', 'MEDUZA;Goodboys', 'A Great Big World', 'MIKTEK', 'Sujatha', 'Seinabo Sey', 'Theo Parrish', 'OAFF;Lothika', 'Tmsoft’s White Noise Sleep Sounds', 'Lasse Stefanz

KeyboardInterrupt: 

In [70]:
import matplotlib.pyplot as plt

# Parameters
num_images_to_display = 40
images_per_row = 4  # Number of images per row for a grid layout
images_displayed = 0

# Collect and display images
plt.figure(figsize=(15, 10))  # Set the figure size

for X, y in forget_loader:
    # Convert the batch to NumPy
    np_images = X.permute(0, 2, 3, 1).numpy()  # Change shape to (N, H, W, C)
    
    # Loop through images in the batch
    for img, label in zip(np_images, y):
        images_displayed += 1
        
        # Add subplot for each image
        plt.subplot(num_images_to_display // images_per_row, images_per_row, images_displayed)
        plt.imshow(img)  # Assuming the range is already suitable for visualization
        plt.title(f"Label: {label.item()}")  # Display label
        plt.axis("off")
        
        if images_displayed == num_images_to_display:
            break  # Stop after displaying the required number of images
    if images_displayed == num_images_to_display:
        break

plt.tight_layout()  # Adjust spacing between subplots
plt.savefig('2820_new_new.png')
plt.show()


RuntimeError: permute(sparse_coo): number of dimensions in the tensor input does not match the length of the desired ordering of dimensions i.e. input.dim() = 2 is not equal to len(dims) = 4

<Figure size 1500x1000 with 0 Axes>

In [11]:
loader, _ = data_manager.get_loader_for('train')

2025-01-31 17:09:43,-1121971173 | INFO | 346018 - TRAINING WITH 124594 samples


In [72]:

from erasure.utils.config.global_ctx import bcolors


#Create Predictor
current = Local(global_ctx.config.predictor)
current.dataset = data_manager
predictor = global_ctx.factory.get_object(current)
global_ctx.predictor = predictor
global_ctx.logger.info('Global Predictor: ' + str(predictor))

#Create unlearners 
unlearners = []
unlearners_cfg = global_ctx.config.unlearners
for un in unlearners_cfg:
    current = Local(un)
    current.dataset = data_manager
    current.predictor = copy.deepcopy(predictor)
    unlearners.append( global_ctx.factory.get_object(current) )


#Evaluator
current = Local(global_ctx.config.evaluator)
current.unlearners = unlearners
evaluator = global_ctx.factory.get_object(current)

# Evaluations
for unlearner in unlearners:
    global_ctx.logger.info(f'''{bcolors.OKGREEN}####\t\t Evaluating: {unlearner.__class__.__name__} \t\t####{bcolors.ENDC}''')
    evaluator.evaluate(unlearner,predictor)

2025-02-01 18:31:30,-1030664600 | INFO | 857580 - Instantiating: erasure.model.classifiers.standard_classifier.AdultNN
2025-02-01 18:31:30,-1030664598 | INFO | 857580 - Instantiating: torch.optim.Adam
2025-02-01 18:31:30,-1030664590 | INFO | 857580 - Instantiating: torch.nn.CrossEntropyLoss
2025-02-01 18:31:30,-1030664589 | INFO | 857580 - TRAINING WITH 39073 samples
2025-02-01 18:31:37,-1030657394 | INFO | 857580 - epoch = 0 ---> loss = 372.0538	 accuracy = 0.6816
2025-02-01 18:31:44,-1030650084 | INFO | 857580 - epoch = 1 ---> loss = 312.5461	 accuracy = 0.6799
2025-02-01 18:31:52,-1030642508 | INFO | 857580 - epoch = 2 ---> loss = 213.9229	 accuracy = 0.6858
2025-02-01 18:32:00,-1030634263 | INFO | 857580 - epoch = 3 ---> loss = 168.1182	 accuracy = 0.6821


KeyboardInterrupt: 